In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim-1.0.1-py3.6-win32.egg\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used


In [7]:
tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()
    
df = pd.read_csv('data/grill-and-bbq-normalize.csv', encoding='utf-8', header=1, sep='|')
recipeSet = [row[5].lower() for row in df.itertuples()]

bow_ingre = []

for i in recipeSet:
    tokens = tokenizer.tokenize(i)
    stemmed_tokens = [p_stemmer.stem(i) for i in tokens]
    bow_ingre.append(stemmed_tokens)

corp_dict = corpora.Dictionary(bow_ingre)
    
corpus = [corp_dict.doc2bow(ingre) for ingre in bow_ingre]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = corp_dict, passes=20)
print(ldamodel.print_topics(num_topics=10, num_words=3))
print(ldamodel.print_topics(num_topics=10, num_words=4))
print(ldamodel.print_topics(num_topics=10, num_words=5))
print(ldamodel.print_topics(num_topics=10, num_words=6))


[(0, '0.083*"onion" + 0.065*"garlic" + 0.059*"sauc"'), (1, '0.093*"pepper" + 0.064*"salt" + 0.060*"oil"')]
[(0, '0.083*"onion" + 0.065*"garlic" + 0.059*"sauc" + 0.045*"sugar"'), (1, '0.093*"pepper" + 0.064*"salt" + 0.060*"oil" + 0.054*"oliv"')]
[(0, '0.083*"onion" + 0.065*"garlic" + 0.059*"sauc" + 0.045*"sugar" + 0.045*"pepper"'), (1, '0.093*"pepper" + 0.064*"salt" + 0.060*"oil" + 0.054*"oliv" + 0.048*"black"')]
[(0, '0.083*"onion" + 0.065*"garlic" + 0.059*"sauc" + 0.045*"sugar" + 0.045*"pepper" + 0.044*"salt"'), (1, '0.093*"pepper" + 0.064*"salt" + 0.060*"oil" + 0.054*"oliv" + 0.048*"black" + 0.048*"chicken"')]
